In [0]:
# REVERSIÓN FÍSICA DE ARCHIVOS (ADLS Gen2)
# Este notebook limpia las rutas físicas donde se almacenan los datos de las tablas Delta
# alineado a la estructura de contenedores por ambiente (dev/prod)

dbutils.widgets.removeAll()
dbutils.widgets.text("storage_account", "adlssmartdata0303ev")
dbutils.widgets.text("entorno", "dev") # <--- Nuevo widget para definir qué ambiente limpiar

storage_account = dbutils.widgets.get("storage_account")
entorno = dbutils.widgets.get("entorno")

# Definición de rutas según la nueva estructura de carpetas MANAGED
# IMPORTANTE: No borramos 'raw/datasets' para no perder los CSVs originales, 
# solo borramos las capas de tablas Delta (bronze, silver, golden).
rutas_a_limpiar = [
    f"abfss://{entorno}@{storage_account}.dfs.core.windows.net/golden/",
    f"abfss://{entorno}@{storage_account}.dfs.core.windows.net/silver/",
    f"abfss://{entorno}@{storage_account}.dfs.core.windows.net/bronze/"
]

for ruta in rutas_a_limpiar:
    try:
        print(f"Intentando eliminar ruta física: {ruta}")
        # recurse=True permite borrar carpetas y subcarpetas. 
        # Es necesario para eliminar los archivos Parquet/Delta dentro de las carpetas.
        dbutils.fs.rm(ruta, recurse=True)
        print(f"Éxito: {ruta} eliminada.")
    except Exception as e:
        print(f"Aviso: No se pudo eliminar {ruta} (Probablemente ya estaba vacía o no existe).")

print(f"Limpieza física del ambiente [{entorno}] completada.")